In [1]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [42]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [43]:
def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [44]:
train_features = pd.read_csv('data/raw/train_features.csv')
train_targets_scored = pd.read_csv('data/raw/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('data/raw/train_targets_nonscored.csv')

test_features = pd.read_csv('data/raw/test_features.csv')
sample_submission = pd.read_csv('data/raw/sample_submission.csv')

In [45]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [46]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

train = train[[col for col in train if col not in target.columns or col == 'sig_id']]

print(train.shape)
print(test.shape)
print(target.shape)

(21948, 876)
(3624, 876)
(21948, 207)


In [47]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

# **CV Folds**

In [48]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)

In [49]:
def process_data(data):
    df = data.copy()
    
    features_g = GENES
    features_c = CELLS
    
    df['g_sum'] = df[features_g].sum(axis = 1)
    df['g_mean'] = df[features_g].mean(axis = 1)
    df['g_std'] = df[features_g].std(axis = 1)
    df['g_kurt'] = df[features_g].kurtosis(axis = 1)
    df['g_skew'] = df[features_g].skew(axis = 1)
    df['c_sum'] = df[features_c].sum(axis = 1)
    df['c_mean'] = df[features_c].mean(axis = 1)
    df['c_std'] = df[features_c].std(axis = 1)
    df['c_kurt'] = df[features_c].kurtosis(axis = 1)
    df['c_skew'] = df[features_c].skew(axis = 1)
    df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
    df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
    df['gc_std'] = df[features_g + features_c].std(axis = 1)
    df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
    df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
    
    for feature in features_c:
        df[f'{feature}_squared'] = df[feature] ** 2
    
    return df


def add_pca(train_df, valid_df, test_):
    
    # GENES
    n_comp = 28

    pca = PCA(n_components=n_comp, random_state=1903)
    pipe = Pipeline([('scal', RobustScaler()), ('pca', pca)])
    train2 = pipe.fit_transform(train_df[GENES])
    valid2 = pipe.transform(valid_df[GENES])
    test2 = pipe.transform(test_[GENES])

    train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
    valid2 = pd.DataFrame(valid2, columns=[f'pca_G-{i}' for i in range(n_comp)])
    test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

    train_df = pd.concat((train_df, train2), axis=1)
    valid_df = pd.concat((valid_df, valid2), axis=1)
    test_ = pd.concat((test_, test2), axis=1)

    #CELLS
    n_comp = 5

    pca = PCA(n_components=n_comp, random_state=1903)
    pipe = Pipeline([('scal', RobustScaler()), ('pca', pca)])
    train2 = pipe.fit_transform(train_df[CELLS])
    valid2 = pipe.transform(valid_df[CELLS])
    test2 = pipe.transform(test_[CELLS])

    train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
    valid2 = pd.DataFrame(valid2, columns=[f'pca_C-{i}' for i in range(n_comp)])
    test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

    train_df = pd.concat((train_df, train2), axis=1)
    valid_df = pd.concat((valid_df, valid2), axis=1)
    test_ = pd.concat((test_, test2), axis=1)
    
    return train_df, valid_df, test_


def var_tr(train_df, valid_df, test_):
    
    var_thresh = VarianceThreshold(threshold=0.9)

    train_transformed = var_thresh.fit_transform(train_df.iloc[:, 4:])
    valid_transformed = var_thresh.transform(valid_df.iloc[:, 4:])
    test_transformed = var_thresh.transform(test_.iloc[:, 4:])

    train_features = pd.DataFrame(train_df[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                                  columns=['sig_id','cp_type','cp_time','cp_dose'])
    train_features = pd.concat([train_features, pd.DataFrame(train_transformed)], axis=1)
    
    valid_features = pd.DataFrame(valid_df[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                                  columns=['sig_id','cp_type','cp_time','cp_dose'])
    valid_features = pd.concat([valid_features, pd.DataFrame(valid_transformed)], axis=1)

    test_features = pd.DataFrame(test_[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                                 columns=['sig_id','cp_type','cp_time','cp_dose'])
    test_features = pd.concat([test_features, pd.DataFrame(test_transformed)], axis=1)
    
    return train_features, valid_features, test_features

# **Utilities**

In [50]:
class DfScaler(BaseEstimator, TransformerMixin):
    '''
    Wrapper of several sklearn scalers that keeps the dataframe structure
    '''
    def __init__(self, method='standard', feature_range=(0,1)):
        super().__init__()
        self.method = method
        self._validate_input()
        self.scale_ = None
        if self.method == 'standard':
            self.scl = StandardScaler()
            self.mean_ = None
        elif method == 'robust':
            self.scl = RobustScaler()
            self.center_ = None
        elif method == 'minmax':
            self.feature_range = feature_range
            self.scl = MinMaxScaler(feature_range=self.feature_range)
            self.min_ = None
            self.data_min_ = None
            self.data_max_ = None
            self.data_range_ = None
            self.n_samples_seen_ = None

            
    def _validate_input(self):
        allowed_methods = ["standard", 'robust', 'minmax']
        if self.method not in allowed_methods:
            raise ValueError(f"Can only use these methods: {allowed_methods} got method={self.method}")
    
    
    def fit(self, X, y=None):
        self.scl.fit(X)
        if self.method == 'standard':
            self.mean_ = pd.Series(self.scl.mean_, index=X.columns)
        elif self.method == 'robust':
            self.center_ = pd.Series(self.scl.center_, index=X.columns)
        elif self.method == 'minmax':
            self.min_ = pd.Series(self.scl.min_, index=X.columns)
            self.data_min_ = pd.Series(self.scl.data_min_, index=X.columns)
            self.data_max_ = pd.Series(self.scl.data_max_, index=X.columns)
            self.data_range_ = self.data_max_ - self.data_min_
            self.n_samples_seen_ = X.shape[0]
        self.scale_ = pd.Series(self.scl.scale_, index=X.columns)
        return self
    
    
    def transform(self, X, y=None):
        # assumes X is a DataFrame
        Xscl = self.scl.transform(X)
        Xscaled = pd.DataFrame(Xscl, index=X.index, columns=X.columns)
        return Xscaled

In [51]:
def scale_data(train, valid, test):
    
    scl = DfScaler(method='robust')
    scaled_train = scl.fit_transform(train[[col for col in train if col!='sig_id']])
    scaled_train = pd.concat([train[['sig_id']], scaled_train], axis=1)
    
    scaled_valid = scl.transform(valid[[col for col in valid if col!='sig_id']])
    scaled_valid = pd.concat([valid[['sig_id']], scaled_valid], axis=1)
    
    scaled_test = scl.transform(test[[col for col in test if col!='sig_id']])
    scaled_test = pd.concat([test[['sig_id']], scaled_test], axis=1)
    
    return scaled_train, scaled_valid, scaled_test

In [52]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

In [53]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        #print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   

In [54]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.25)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.25)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.25)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x), 1e-3)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [55]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 40
BATCH_SIZE = 256
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-5
NFOLDS = 7
EARLY_STOPPING_STEPS = 5
EARLY_STOP = True

hidden_size=2048

In [56]:
def run_training(train, test, fold, seed):
    
    test_ = test.copy()
    
    seed_everything(seed)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    del train_df['kfold']
    del valid_df['kfold']
    
    train_df, valid_df, test_ = add_pca(train_df, valid_df, test_)
    
    train_df = process_data(train_df)
    valid_df = process_data(valid_df)
    test_ = process_data(test_)
    
    train_df, valid_df, test_ = var_tr(train_df, valid_df, test_)
    
    train_df = train_df.drop('cp_type', axis=1)
    valid_df = valid_df.drop('cp_type', axis=1)
    test_ = test_.drop('cp_type', axis=1)
    
    train_df['time_dose'] = train_df['cp_time'].astype(str)+train_df['cp_dose']
    valid_df['time_dose'] = valid_df['cp_time'].astype(str)+valid_df['cp_dose']
    test_['time_dose'] = test_['cp_time'].astype(str)+test_['cp_dose']
    
    train_df = pd.get_dummies(train_df, columns=['cp_time','cp_dose','time_dose'])
    valid_df = pd.get_dummies(valid_df, columns=['cp_time','cp_dose','time_dose'])
    test_ = pd.get_dummies(test_, columns=['cp_time','cp_dose','time_dose'])
    
    feature_cols = [c for c in train_df.columns if c not in target_cols]
    feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
    num_features=len(feature_cols)
    num_targets=len(target_cols)
    
    #scaling
    train_df, valid_df, test_ = scale_data(train_df, valid_df, test_)
    
    x_train, y_train  = train_df[feature_cols].values, target.iloc[trn_idx, :][target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, target.iloc[val_idx, :][target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    print(f"FOLD: {fold}, n_features={num_features}")
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
            early_step = 0
        
        else:
            if EARLY_STOP:
                early_step += 1
                if (early_step >= early_stopping_steps):
                    break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [57]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(folds, test, fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [ ]:
# Averaging on multiple SEEDS

SEED = [1903, 1881] # , 346, 4578, 21
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    print(f"SEED: {seed}")
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

SEED: 1903
FOLD: 0, n_features=906
EPOCH: 0, train_loss: 0.6891456222211992, valid_loss: 0.4691091019373674
EPOCH: 1, train_loss: 0.10199048214063451, valid_loss: 0.0206038963336211
EPOCH: 2, train_loss: 0.01986665532899064, valid_loss: 0.02000745987662902
EPOCH: 3, train_loss: 0.018495037421785498, valid_loss: 0.018139184094392337
EPOCH: 4, train_loss: 0.017817321082426084, valid_loss: 0.01784716138186363
EPOCH: 5, train_loss: 0.017339889551638753, valid_loss: 0.017402811262470026
EPOCH: 6, train_loss: 0.01710958017439053, valid_loss: 0.017176977573679045
EPOCH: 7, train_loss: 0.016944393062511005, valid_loss: 0.017100217537238047
EPOCH: 8, train_loss: 0.016995989063100236, valid_loss: 0.017052864727492515
EPOCH: 9, train_loss: 0.016964867207649593, valid_loss: 0.017271751108077858
EPOCH: 10, train_loss: 0.017023750228454936, valid_loss: 0.0169291110136188
EPOCH: 11, train_loss: 0.017008958925568574, valid_loss: 0.016999345201139267
EPOCH: 12, train_loss: 0.016989293822867645, valid_l

In [41]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.015347327538593266


In [22]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)